In [2]:
!pwd

/home/oleg/projects/ML/OWL-ViT


In [2]:


# Автоматическая разметка папки с изображениями

import os
import json
from pathlib import Path
from PIL import Image, ImageDraw
import torch
from transformers import OwlViTProcessor, OwlViTForObjectDetection
import matplotlib.pyplot as plt

def quick_batch_process(input_folder, output_folder, what_to_find):
    """Упрощенная версия для быстрой обработки"""
    
    from pathlib import Path
    Path(output_folder).mkdir(exist_ok=True)
    
    processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
    model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")
    
    for image_file in os.listdir(input_folder):
        if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_folder, image_file)
            base_name = os.path.splitext(image_file)[0]
            
            # Обработка
            image = Image.open(input_path)
            texts = [[what_to_find]]
            inputs = processor(text=texts, images=image, return_tensors="pt")
            outputs = model(**inputs)
            
            # Сохраняем результат
            target_sizes = torch.Tensor([image.size[::-1]])
            results = processor.post_process_object_detection(
                outputs=outputs, target_sizes=target_sizes, threshold=0.2
            )
            
            # Сохраняем JSON
            output_json = {
                "file": image_file,
                "objects_found": len([s for s in results[0]["scores"] if s > 0.2]),
                "prompt": what_to_find
            }
            
            json_path = os.path.join(output_folder, f"{base_name}.json")
            with open(json_path, 'w') as f:
                json.dump(output_json, f, indent=2)
            
            print(f"✅ {image_file} -> {json_path}")

# Быстрый запуск
quick_batch_process(
    "/home/oleg/projects/ML/OWL-ViT/images",
    "/home/oleg/projects/ML/OWL-ViT/processed", 
    "a person, a cat, a car"
)

✅ 3c237a92b724ac939f84493963c00952.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/3c237a92b724ac939f84493963c00952.json
✅ gettyimages-524401415-1024x1024.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/gettyimages-524401415-1024x1024.json
✅ 05a992e228da2abf2d571db367a3febf.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/05a992e228da2abf2d571db367a3febf.json
✅ elegant-portrait-black-cat-indoors_23-2151890796.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/elegant-portrait-black-cat-indoors_23-2151890796.json
✅ cat-pictures-zc3gu0636kmldm04.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/cat-pictures-zc3gu0636kmldm04.json
✅ 409052_size1.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/409052_size1.json
✅ 1e64d77677f5ce8714d552f30f2422bb.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/1e64d77677f5ce8714d552f30f2422bb.json
✅ 0d354ad89e92986b19b10a8ac2797dfb.jpg -> /home/oleg/projects/ML/OWL-ViT/processed/0d354ad89e92986b19b10a8ac2797dfb.json


In [1]:
import os
import json
from pathlib import Path
from PIL import Image, ImageDraw
import torch
from transformers import OwlViTProcessor, OwlViTForObjectDetection
import matplotlib.pyplot as plt

def process_folder_with_owlvit(input_folder, output_folder, text_prompts):
    """
    Обрабатывает все изображения в папке и сохраняет результаты
    
    Args:
        input_folder: папка с исходными изображениями
        output_folder: папка для сохранения результатов
        text_prompts: список объектов для поиска через запятую
    """
    
    # Создаем выходную папку если нет
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # Загружаем модель
    print("🦉 Загружаем Owl-ViT...")
    processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
    model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")
    
    # Получаем список изображений
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
    image_files = [f for f in os.listdir(input_folder) 
                  if os.path.splitext(f)[1].lower() in image_extensions]
    
    print(f"📁 Найдено {len(image_files)} изображений в {input_folder}")
    
    for image_file in image_files:
        try:
            input_path = os.path.join(input_folder, image_file)
            base_name = os.path.splitext(image_file)[0]
            
            print(f"🔍 Обрабатываю: {image_file}")
            
            # Обрабатываем изображение
            result = process_single_image(
                input_path, text_prompts, processor, model
            )
            
            # Сохраняем результаты
            save_results(output_folder, base_name, result, input_path)
            
            print(f"✅ Готово: {image_file}")
            
        except Exception as e:
            print(f"❌ Ошибка с {image_file}: {e}")

def process_single_image(image_path, text_prompts, processor, model):
    """Обрабатывает одно изображение и возвращает результаты"""
    
    # Загружаем изображение
    image = Image.open(image_path).convert('RGB')
    
    # Разделяем текстовые промпты
    texts = [text_prompts.split(",")]
    
    # Обработка моделью
    inputs = processor(text=texts, images=image, return_tensors="pt")
    outputs = model(**inputs)
    
    # Получаем детекции
    target_sizes = torch.Tensor([image.size[::-1]])
    results = processor.post_process_object_detection(
        outputs=outputs, target_sizes=target_sizes, threshold=0.1
    )
    
    # Формируем результат
    detections = []
    for box, score, label in zip(results[0]["boxes"], results[0]["scores"], results[0]["labels"]):
        if score > 0.2:  # Фильтр по уверенности
            x1, y1, x2, y2 = box.tolist()
            detection = {
                "label": texts[0][label],
                "confidence": float(score),
                "bbox": [x1, y1, x2, y2],
                "bbox_normalized": [  # Нормализованные координаты (0-1)
                    x1 / image.width, 
                    y1 / image.height, 
                    x2 / image.width, 
                    y2 / image.height
                ]
            }
            detections.append(detection)
    
    return {
        "image_path": image_path,
        "image_size": [image.width, image.height],
        "text_prompts": text_prompts,
        "detections": detections,
        "total_detections": len(detections)
    }

def save_results(output_folder, base_name, result, original_image_path):
    """Сохраняет все результаты для одного изображения"""
    
    # 1. Сохраняем JSON с описанием
    json_path = os.path.join(output_folder, f"{base_name}_detections.json")
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)
    
    # 2. Создаем и сохраняем изображение с разметкой
    image_with_boxes = create_image_with_boxes(original_image_path, result)
    image_output_path = os.path.join(output_folder, f"{base_name}_annotated.jpg")
    image_with_boxes.save(image_output_path, quality=95)
    
    # 3. Создаем текстовый отчет
    report_path = os.path.join(output_folder, f"{base_name}_report.txt")
    create_text_report(report_path, result)

def create_image_with_boxes(image_path, result):
    """Создает изображение с bounding boxes"""
    
    image = Image.open(image_path).convert('RGB')
    draw = ImageDraw.Draw(image)
    
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
    
    for i, detection in enumerate(result['detections']):
        color = colors[i % len(colors)]
        x1, y1, x2, y2 = detection['bbox']
        
        # Рисуем bounding box
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
        
        # Рисуем подпись
        label = f"{detection['label']} ({detection['confidence']:.2f})"
        draw.rectangle([x1, y1-25, x1+len(label)*8, y1], fill=color)
        draw.text((x1+5, y1-20), label, fill='white')
    
    return image

def create_text_report(report_path, result):
    """Создает текстовый отчет"""
    
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("=" * 50 + "\n")
        f.write("ОТЧЕТ О ДЕТЕКЦИИ ОБЪЕКТОВ\n")
        f.write("=" * 50 + "\n\n")
        
        f.write(f"📁 Изображение: {os.path.basename(result['image_path'])}\n")
        f.write(f"📐 Размер: {result['image_size'][0]}x{result['image_size'][1]}\n")
        f.write(f"🔍 Запрос: {result['text_prompts']}\n")
        f.write(f"📊 Найдено объектов: {result['total_detections']}\n\n")
        
        f.write("🎯 ОБНАРУЖЕННЫЕ ОБЪЕКТЫ:\n")
        f.write("-" * 30 + "\n")
        
        for i, detection in enumerate(result['detections'], 1):
            f.write(f"{i}. {detection['label']}\n")
            f.write(f"   Уверенность: {detection['confidence']:.3f}\n")
            f.write(f"   BBox: [{detection['bbox'][0]:.1f}, {detection['bbox'][1]:.1f}, "
                   f"{detection['bbox'][2]:.1f}, {detection['bbox'][3]:.1f}]\n")
            f.write(f"   BBox (норм.): [{detection['bbox_normalized'][0]:.3f}, "
                   f"{detection['bbox_normalized'][1]:.3f}, {detection['bbox_normalized'][2]:.3f}, "
                   f"{detection['bbox_normalized'][3]:.3f}]\n\n")

# 🎯 ИСПОЛЬЗОВАНИЕ:

if __name__ == "__main__":
    # Настройки

    
    INPUT_FOLDER = "/home/oleg/projects/ML/OWL-ViT/images"  # Папка с исходными изображениями
    OUTPUT_FOLDER = "/home/oleg/projects/ML/OWL-ViT/processed"  # Папка для результатов
    TEXT_PROMPTS = "a person, a cat, a car"  # Что ищем
    
    print("🚀 ЗАПУСК ПАКЕТНОЙ ОБРАБОТКИ...")
    print(f"📁 Входная папка: {INPUT_FOLDER}")
    print(f"📁 Выходная папка: {OUTPUT_FOLDER}")
    print(f"🔍 Поиск объектов: {TEXT_PROMPTS}")
    print("=" * 60)
    
    process_folder_with_owlvit(INPUT_FOLDER, OUTPUT_FOLDER, TEXT_PROMPTS)
    
    print("🎉 ОБРАБОТКА ЗАВЕРШЕНА!")
    print(f"📊 Результаты сохранены в: {OUTPUT_FOLDER}")

2025-10-31 18:37:18.669768: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-31 18:37:18.711214: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-31 18:37:20.366316: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


🚀 ЗАПУСК ПАКЕТНОЙ ОБРАБОТКИ...
📁 Входная папка: /home/oleg/projects/ML/OWL-ViT/images
📁 Выходная папка: /home/oleg/projects/ML/OWL-ViT/processed
🔍 Поиск объектов: a person, a cat, a car
🦉 Загружаем Owl-ViT...
📁 Найдено 8 изображений в /home/oleg/projects/ML/OWL-ViT/images
🔍 Обрабатываю: 3c237a92b724ac939f84493963c00952.jpg


/home/oleg/miniconda3/envs/tensflow/lib/python3.11/site-packages/transformers/models/owlvit/processing_owlvit.py:217: FutureWarning: `post_process_object_detection` method is deprecated for OwlVitProcessor and will be removed in v5. Use `post_process_grounded_object_detection` instead.
  warnings.warn(


✅ Готово: 3c237a92b724ac939f84493963c00952.jpg
🔍 Обрабатываю: gettyimages-524401415-1024x1024.jpg
✅ Готово: gettyimages-524401415-1024x1024.jpg
🔍 Обрабатываю: 05a992e228da2abf2d571db367a3febf.jpg
✅ Готово: 05a992e228da2abf2d571db367a3febf.jpg
🔍 Обрабатываю: elegant-portrait-black-cat-indoors_23-2151890796.jpg
✅ Готово: elegant-portrait-black-cat-indoors_23-2151890796.jpg
🔍 Обрабатываю: cat-pictures-zc3gu0636kmldm04.jpg
✅ Готово: cat-pictures-zc3gu0636kmldm04.jpg
🔍 Обрабатываю: 409052_size1.jpg
✅ Готово: 409052_size1.jpg
🔍 Обрабатываю: 1e64d77677f5ce8714d552f30f2422bb.jpg
✅ Готово: 1e64d77677f5ce8714d552f30f2422bb.jpg
🔍 Обрабатываю: 0d354ad89e92986b19b10a8ac2797dfb.jpg
✅ Готово: 0d354ad89e92986b19b10a8ac2797dfb.jpg
🎉 ОБРАБОТКА ЗАВЕРШЕНА!
📊 Результаты сохранены в: /home/oleg/projects/ML/OWL-ViT/processed
